<a href="https://colab.research.google.com/github/Piyumi22/XAI/blob/main/SHAP_example_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
df.head()